In [1]:
#Making necesarry imports
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import correlation
from sklearn.metrics.pairwise import pairwise_distances
import ipywidgets as widgets
from IPython.display import display, clear_output
from contextlib import contextmanager
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import os, sys
import re
import seaborn as sns

In [2]:
goods = pd.read_csv('./qf_goods.csv', header=None)
goods.columns = ['id', 'is_delete', 'title', 'price', 'clicks', 'collections', 'praises', 'bads', 'state', 'tag_id', 'sales']
goods.set_index('id')
goods=goods.drop(['title'],axis=1)

In [3]:
# 删除 商品不在出售的
goods.drop(goods[goods.is_delete == True].index, inplace=True)
goods.drop(goods[goods.state != 1].index, inplace=True)
goods

,id,is_delete,price,clicks,collections,praises,bads,state,tag_id,sales
31,32,0,938,7327,2586,3429,919,1,1,5025
32,33,0,2399,5544,1790,2250,1224,1,1,3001
33,34,0,668,2751,619,1067,261,1,1,1991
34,35,0,4999,622,283,316,222,1,1,483
35,36,0,5699,112,33,61,7,1,1,71
...,...,...,...,...,...,...,...,...,...,...
223,224,0,3,6563,1849,3584,256,1,14,3996
224,225,0,5,3883,588,1262,852,1,14,2007
225,226,0,16,1629,218,409,110,1,14,994
226,227,0,12,1101,353,626,218,1,14,1043


In [16]:
goods.eval('score1 =(collections + praises - bads*3 + sales)/clicks', inplace=True)
goods

,id,is_delete,price,clicks,collections,praises,bads,state,tag_id,sales,score,price_cut,clicks_cut,collections_cut,praises_cut,bads_cut,sales_cut,score1
31,32,0,938,7327,2586,3429,919,1,1,5025,15610,7,8,8,8,7,7,1.130476
32,33,0,2399,5544,1790,2250,1224,1,1,3001,8913,8,7,7,6,7,6,0.607684
33,34,0,668,2751,619,1067,261,1,1,1991,5645,7,6,4,5,4,5,1.051981
34,35,0,4999,622,283,316,222,1,1,483,1038,8,3,2,2,4,2,0.668810
35,36,0,5699,112,33,61,7,1,1,71,256,9,1,0,0,0,0,1.285714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,224,0,3,6563,1849,3584,256,1,14,3996,15224,0,8,7,8,4,7,1.319671
224,225,0,5,3883,588,1262,852,1,14,2007,5184,0,6,4,5,6,5,0.335050
225,226,0,16,1629,218,409,110,1,14,994,2920,1,5,2,2,2,4,0.792511
226,227,0,12,1101,353,626,218,1,14,1043,2469,0,5,3,3,4,4,1.242507


In [20]:
goods['score1'].min()

0.04

In [5]:
goods['price_cut'] = pd.qcut(goods.price, q=10, labels=[i for i in range(10)])
goods

,id,is_delete,price,clicks,collections,praises,bads,state,tag_id,sales,score,price_cut
31,32,0,938,7327,2586,3429,919,1,1,5025,15610,7
32,33,0,2399,5544,1790,2250,1224,1,1,3001,8913,8
33,34,0,668,2751,619,1067,261,1,1,1991,5645,7
34,35,0,4999,622,283,316,222,1,1,483,1038,8
35,36,0,5699,112,33,61,7,1,1,71,256,9
...,...,...,...,...,...,...,...,...,...,...,...,...
223,224,0,3,6563,1849,3584,256,1,14,3996,15224,0
224,225,0,5,3883,588,1262,852,1,14,2007,5184,0
225,226,0,16,1629,218,409,110,1,14,994,2920,1
226,227,0,12,1101,353,626,218,1,14,1043,2469,0


In [6]:
pd.cut(goods.clicks, bins=[0, 100, 300, 500, 800, 1000, 2000, 4000, 6000, 10000, 100000], labels=[i for i in range(10)])
goods['clicks_cut'] = pd.cut(goods.clicks, bins=[0, 100, 300, 500, 800, 1000, 2000, 4000, 6000, 10000, 100000], labels=[i for i in range(10)])

In [7]:
goods['collections_cut'] = pd.qcut(goods.collections, q=10, labels=[i for i in range(10)])
goods['praises_cut'] = pd.qcut(goods.praises, q=10, labels=[i for i in range(10)])
goods['bads_cut'] = pd.qcut(goods.bads, q=10, labels=[i for i in range(10)])
goods['sales_cut'] = pd.qcut(goods.sales, q=10, labels=[i for i in range(10)])
goods

,id,is_delete,price,clicks,collections,praises,bads,state,tag_id,sales,score,price_cut,clicks_cut,collections_cut,praises_cut,bads_cut,sales_cut
31,32,0,938,7327,2586,3429,919,1,1,5025,15610,7,8,8,8,7,7
32,33,0,2399,5544,1790,2250,1224,1,1,3001,8913,8,7,7,6,7,6
33,34,0,668,2751,619,1067,261,1,1,1991,5645,7,6,4,5,4,5
34,35,0,4999,622,283,316,222,1,1,483,1038,8,3,2,2,4,2
35,36,0,5699,112,33,61,7,1,1,71,256,9,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,224,0,3,6563,1849,3584,256,1,14,3996,15224,0,8,7,8,4,7
224,225,0,5,3883,588,1262,852,1,14,2007,5184,0,6,4,5,6,5
225,226,0,16,1629,218,409,110,1,14,994,2920,1,5,2,2,2,4
226,227,0,12,1101,353,626,218,1,14,1043,2469,0,5,3,3,4,4


In [8]:
d_goods = goods[['id', 'price_cut', 'clicks_cut', 'collections_cut', 'praises_cut', 'bads_cut', 'sales_cut']]
d_goods

,id,price_cut,clicks_cut,collections_cut,praises_cut,bads_cut,sales_cut
31,32,7,8,8,8,7,7
32,33,8,7,7,6,7,6
33,34,7,6,4,5,4,5
34,35,8,3,2,2,4,2
35,36,9,1,0,0,0,0
...,...,...,...,...,...,...,...
223,224,0,8,7,8,4,7
224,225,0,6,4,5,6,5
225,226,1,5,2,2,2,4
226,227,0,5,3,3,4,4


In [9]:
n_goods = d_goods.shape[0] #considering only those users who gave explicit ratings
n_attribute = d_goods.shape[1]
print(n_goods, n_attribute)

196 7


In [10]:
#setting global variables
global metric,k
k=10
metric='cosine'

In [12]:

def findksimilargoods(good_id, ratings, metric = metric, k=k):
    similarities=[]
    indices=[]
    model_knn = NearestNeighbors(metric = metric, algorithm = 'brute') 
    model_knn.fit(ratings)
    loc = ratings.index.get_loc(good_id)
    distances, indices = model_knn.kneighbors(ratings.iloc[loc, :].values.reshape(1, -1), n_neighbors = k+1)
    similarities = 1-distances.flatten()
            
    return similarities,indices

In [13]:
print(findksimilarusers(32, d_goods))

(array([1.        , 0.99671472, 0.99268461, 0.9916296 , 0.98383698,
       0.9837275 , 0.97966374, 0.97922875, 0.97843104, 0.97830125,
       0.97812385]), array([[ 1,  0, 18,  2, 23, 29, 21, 31, 25, 34, 16]], dtype=int64))


In [18]:
pd.DataFrame(cosine_sim)

,0,1,2,3,4,5,6,7,8,9,...,186,187,188,189,190,191,192,193,194,195
0,1.000000,0.996715,0.982983,0.947654,0.892388,0.902450,0.924604,0.911499,0.896623,0.914767,...,0.879441,0.867619,0.872333,0.873136,0.899825,0.896168,0.889271,0.880803,0.883306,0.882380
1,0.996715,1.000000,0.991630,0.968265,0.921457,0.929232,0.949075,0.937463,0.925265,0.939489,...,0.902431,0.891889,0.895729,0.896532,0.919318,0.915588,0.910314,0.903221,0.905243,0.904687
2,0.982983,0.991630,1.000000,0.987543,0.957622,0.964314,0.976996,0.969068,0.960338,0.971831,...,0.943462,0.935623,0.938413,0.938863,0.954869,0.952959,0.948861,0.944355,0.945333,0.945149
3,0.947654,0.968265,0.987543,1.000000,0.987609,0.989879,0.996877,0.993318,0.990044,0.993688,...,0.966963,0.962150,0.963071,0.963528,0.970976,0.969125,0.968386,0.966362,0.966628,0.967191
4,0.892388,0.921457,0.957622,0.987609,1.000000,0.999291,0.996416,0.998588,0.999420,0.997278,...,0.972029,0.970882,0.969955,0.969925,0.967774,0.968420,0.969189,0.970946,0.969678,0.970847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,0.896168,0.915588,0.952959,0.969125,0.968420,0.971019,0.971816,0.974314,0.973990,0.981363,...,0.998870,0.997587,0.998362,0.998409,0.999728,1.000000,0.999699,0.999158,0.999401,0.999229
192,0.889271,0.910314,0.948861,0.968386,0.969189,0.971207,0.971674,0.974430,0.975002,0.981309,...,0.999583,0.998630,0.999201,0.999288,0.999639,0.999699,1.000000,0.999683,0.999887,0.999815
193,0.880803,0.903221,0.944355,0.966362,0.970946,0.972472,0.971560,0.975008,0.976240,0.981689,...,0.999905,0.999482,0.999790,0.999798,0.998734,0.999158,0.999683,1.000000,0.999933,0.999943
194,0.883306,0.905243,0.945333,0.966628,0.969678,0.971344,0.971012,0.974253,0.975287,0.981060,...,0.999863,0.999263,0.999669,0.999717,0.999155,0.999401,0.999887,0.999933,1.000000,0.999971


In [19]:
def findksimilarusers(good_id, ratings, metric = metric, k=k):
    similarities=[]
    indices=[]
    model_knn = NearestNeighbors(metric = metric, algorithm = 'brute') 
    model_knn.fit(ratings)

    distances, indices = model_knn.kneighbors(ratings.iloc[good_id-1, :].values.reshape(1, -1), n_neighbors = k+1)
    similarities = 1-distances.flatten()
    print('{0} most similar users for User {1}:\n'.format(k,good_id))
    for i in range(0, len(indices.flatten())):
        if indices.flatten()[i]+1 == good_id:
            continue;

        else:
            print('{0}: Goods {1}, with similarity of {2}'.format(i, indices.flatten()[i]+1, similarities.flatten()[i]))
            
    return similarities,indices

In [20]:
similarities,indices = findksimilarusers(32, d_goods, metric='cosine')

10 most similar users for User 32:

1: User 23, with similarity of 0.9987825932585425
2: User 24, with similarity of 0.9984318048556797
3: User 25, with similarity of 0.9978088132025005
4: User 26, with similarity of 0.9977720640532853
5: User 30, with similarity of 0.997405493994437
6: User 22, with similarity of 0.9973280716090517
7: User 57, with similarity of 0.9972103504311793
8: User 20, with similarity of 0.9967144310665146
9: User 18, with similarity of 0.9961452301356504
10: User 33, with similarity of 0.9959769887900451
